In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Определение параметров модели и данных
IMG_WIDTH = 150  # Ширина изображения
IMG_HEIGHT = 150  # Высота изображения
NUM_CLASSES = 2  # Количество классов (кошки и собаки)
BATCH_SIZE = 32  # Размер пакета (batch)
EPOCHS = 25  # Количество эпох (проходов по данным)

# Папки с данными для обучения и валидации
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/validation'
testing_data_dir = 'dataset/test'

# Аугментация изображений для обучающего набора данных
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Масштабирование значений пикселей в диапазоне [0, 1]
    rotation_range=20,  # Поворот изображения на угол до 20 градусов
    zoom_range=0.2,  # Изменение масштаба изображения до 20%
    width_shift_range=0.2,  # Сдвиг изображения по горизонтали до 20%
    height_shift_range=0.2,  # Сдвиг изображения по вертикали до 20%
    shear_range=0.2,  # Деформация изображения до 20%
    horizontal_flip=True,  # Горизонтальное зеркальное отражение изображения
    fill_mode='nearest',  # Способ заполнения пустых областей при преобразованиях
    validation_split=0.2  # Отделение 20% данных для валидации
)

# Аугментация изображений для валидационного набора данных (без преобразований)
test_datagen = ImageDataGenerator(rescale=1./255)

# Генераторы изображений для обучающего, валидационного и тестового наборов данных
train_generator = train_datagen.flow_from_directory(
    train_data_dir,  # Директория с обучающими данными
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # Размер изображения
    batch_size=BATCH_SIZE,  # Размер пакета (batch)
    class_mode='categorical',  # Формат выходных данных (one-hot encoding)
    subset='training'  # Использование только обучающего набора данных
)

validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,  # Директория с валидационными данными
    target_size=(IMG_WIDTH, IMG_HEIGHT),  # Размер изображения
    batch_size=BATCH_SIZE,  # Размер пакета (batch)
    class_mode='categorical',  # Формат выходных данных (one-hot encoding)
    subset='validation'  # Использование только валидационного набора данных
)
testing_generator = test_datagen.flow_from_directory(
    testing_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


# Создание модели
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))  # Использование предобученной модели MobileNet без верхних слоев

    for layer in base_model.layers[:]:  # Замораживание весов предобученной модели
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))  # Входной слой
    custom_model = base_model(input)  # Подключение предобученной модели
    custom_model = GlobalAveragePooling2D()(custom_model)  # Добавление слоя глобального среднего пулинга
    custom_model = Dense(64, activation='relu')(custom_model)  # Добавление промежуточного полносвязного слоя
    custom_model = Dropout(0.5)(custom_model) # Добавление слоя Dropout для предотвращения переобучения
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model) # Добавление выходного слоя с softmax-активацией

    model = Model(inputs=input, outputs=predictions) # Создание модели Keras

    return model

# Обучение модели
model = model_maker()  # Создание модели
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy', 'val_accuracy'])  # Компиляция модели с указанием функции потерь, оптимизатора и метрик

history = model.fit_generator(
    train_generator,  # Генератор обучающих данных
    validation_data=validation_generator,  # Генератор валидационных данных
    epochs=EPOCHS,  # Количество эпох (проходов по данным)
    steps_per_epoch=train_generator.samples // BATCH_SIZE,  # Количество шагов (пакетов) в эпохе для обучающих данных
    validation_steps=validation_generator.samples // BATCH_SIZE  # Количество шагов (пакетов) в эпохе для валидационных данных
)

testing_steps = testing_generator.samples // BATCH_SIZE
test_loss, test_acc = model.evaluate_generator(testing_generator, steps=testing_steps)
print(f'Тестовая точность: {test_acc:.4f}')

# Вывод итоговой точности на валидационном наборе данных
val_acc = history.history['val_accuracy'][-1]
print(f'Итоговая точность на валидационном наборе данных: {val_acc:.4f}')


Found 3843 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


17225924/17225924 [==============================] - 1s 0us/step


<ipython-input-7-41f3d02cefad>:82: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
120/120 [==============================] - 107s 867ms/step - loss: 0.2995 - accuracy: 0.8845 - val_loss: 0.1454 - val_accuracy: 0.9406
Epoch 2/25
120/120 [==============================] - 103s 859ms/step - loss: 0.1763 - accuracy: 0.9271 - val_loss: 0.0966 - val_accuracy: 0.9625
Epoch 3/25
120/120 [==============================] - 102s 844ms/step - loss: 0.1488 - accuracy: 0.9368 - val_loss: 0.1410 - val_accuracy: 0.9500
Epoch 4/25
120/120 [==============================] - 102s 850ms/step - loss: 0.1382 - accuracy: 0.9441 - val_loss: 0.1329 - val_accuracy: 0.9531
Epoch 5/25
120/120 [==============================] - 103s 852ms/step - loss: 0.1241 - accuracy: 0.9494 - val_loss: 0.0971 - val_accuracy: 0.9625
Epoch 6/25
120/120 [==============================] - 100s 830ms/step - loss: 0.1329 - accuracy: 0.9483 - val_loss: 0.1004 - val_accuracy: 0.9625
Epoch 7/25
120/120 [==============================] - 101s 838ms/step - loss: 0.1204 - accuracy: 0.9480 - val_loss: 0.0980 -

<ipython-input-7-41f3d02cefad>:91: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(testing_generator, steps=testing_steps)


Тестовая точность: 0.9675
Итоговая точность на валидационном наборе данных: 0.9625
